In [1]:
import pandas as pd
import json
from collections import defaultdict
import re
import dataExtraction2

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados. PRUEBA

Si se han usado los pasos minimos:

In [2]:
def extraerDatosNivelCompletado(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]
            
            if erInteracted.search(verb):
                try:
                    dataExtraction2.anadirAlDiccionario(data, evento, name)
                except KeyError:
                    pass

    return data            

In [3]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data = extraerDatosNivelCompletado(rawData)
pd.DataFrame(data)

JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

pd.DataFrame(data)

,gqoj,hgfx,qeqyj,kmqou,loemd,eprdu,gcpze,uzzpo,gleqe,ftpnb,srjon,vsxld,sxipu,fkmtp,xnpqf,ipdcp,ejeuw,epqkm,khadt,ikcar,zxvkj,bdwyj,zwzfv,hjpbw,pbgky,pygoc,tjfzy,yvpmf,asopk,cmbfi,krahz,ubmtx,auvqa,oahey,dunul,kkoif,zcibf,bhyhw,rdiaf,dwqwi,grkij,onyep,yfohf,nmveq,edxmq,swvxx,tekwp,accul,sjpkc,psido,dvmlu,mymho,mjiso,lwryl,gguuc,wjaxi,ucdvy,ewqbs,fivin,tgwmn,camhi,hqmhq,hpdxj,gdljy,fyyya,zbgwk,tinco,kkliq,yxewz,ztyed,rziia,yxjji,dmwvy,tqgyp,qfsdj,ngavz,aotlm,atlzs,mgaak,stiry,yazod,zdnso,lhrbj,bjvnt,rqlug,huyyb,tavin,vvqlm,wqmgq,vddlg,idmzi,tvxow,jhbuv,knewj,jozyx,ahxym,xtdrq,pbwkf,nuasx,itkas,hcbyl,gzgfk,buvgs,ptmkr,fmtrq,thaho,rqaby,kzles,mewpz,uniee,yyvas,icllt,pzfdd,nhbnj,rkmrw,qwnpf,khfha,nplrf,sbexy,xbmmw,ixrux,pqwaf,raqdt,qgcgc,fbldj,lohyo,qbsjv,pleui,tflzv,fmupf,sybwc,eayrh,prpej,blvkx,orjyn,rsjxu,rfofs,tcvyz,bajbm,cojqb,vkfnt,olugf,esrit,epjcx
0,"{'tarjetasCreadas': 11, 'tarjetasMovidas': 9, 'tarjetasEliminadas': 4}","{'tarjetasCreadas': 3, 'tarjetasMovidas': 3, 'tarjetasEliminadas': 0}","{'tarjetasCreadas': 155, 'tarjetasMovidas': 40, 'tarjetasEliminadas': 27}","{'tarjetasCreadas': 111, 'tarjetasMovidas': 32, 'tarjetasEliminadas': 40}","{'tarjetasCreadas': 153, 'tarjetasMovidas': 31, 'tarjetasEliminadas': 9}","{'tarjetasCreadas': 210, 'tarjetasMovidas': 40, 'tarjetasEliminadas': 17}","{'tarjetasCreadas': 214, 'tarjetasMovidas': 41, 'tarjetasEliminadas': 23}","{'tarjetasCreadas': 171, 'tarjetasMovidas': 13, 'tarjetasEliminadas': 17}","{'tarjetasCreadas': 161, 'tarjetasMovidas': 48, 'tarjetasEliminadas': 9}","{'tarjetasCreadas': 209, 'tarjetasMovidas': 61, 'tarjetasEliminadas': 8}","{'tarjetasCreadas': 155, 'tarjetasMovidas': 30, 'tarjetasEliminadas': 13}","{'tarjetasCreadas': 139, 'tarjetasMovidas': 14, 'tarjetasEliminadas': 9}","{'tarjetasCreadas': 75, 'tarjetasMovidas': 16, 'tarjetasEliminadas': 20}","{'tarjetasCreadas': 116, 'tarjetasMovidas': 13, 'tarjetasEliminadas': 1}","{'tarjetasCreadas': 118, 'tarjetasMovidas': 11, 'tarjetasEliminadas': 7}","{'tarjetasCreadas': 205, 'tarjetasMovidas': 41, 'tarjetasEliminadas': 25}","{'tarjetasCreadas': 103, 'tarjetasMovidas': 30, 'tarjetasEliminadas': 18}","{'tarjetasCreadas': 34, 'tarjetasMovidas': 13, 'tarjetasEliminadas': 11}","{'tarjetasCreadas': 156, 'tarjetasMovidas': 37, 'tarjetasEliminadas': 16}","{'tarjetasCreadas': 107, 'tarjetasMovidas': 31, 'tarjetasEliminadas': 18}","{'tarjetasCreadas': 130, 'tarjetasMovidas': 48, 'tarjetasEliminadas': 25}","{'tarjetasCreadas': 98, 'tarjetasMovidas': 84, 'tarjetasEliminadas': 11}","{'tarjetasCreadas': 102, 'tarjetasMovidas': 23, 'tarjetasEliminadas': 10}","{'tarjetasCreadas': 84, 'tarjetasMovidas': 32, 'tarjetasEliminadas': 4}","{'tarjetasCreadas': 161, 'tarjetasMovidas': 35, 'tarjetasEliminadas': 16}","{'tarjetasCreadas': 179, 'tarjetasMovidas': 45, 'tarjetasEliminadas': 13}","{'tarjetasCreadas': 118, 'tarjetasMovidas': 25, 'tarjetasEliminadas': 31}","{'tarjetasCreadas': 192, 'tarjetasMovidas': 86, 'tarjetasEliminadas': 39}","{'tarjetasCreadas': 252, 'tarjetasMovidas': 42, 'tarjetasEliminadas': 9}","{'tarjetasCreadas': 200, 'tarjetasMovidas': 6, 'tarjetasEliminadas': 18}","{'tarjetasCreadas': 157, 'tarjetasMovidas': 24, 'tarjetasEliminadas': 8}","{'tarjetasCreadas': 158, 'tarjetasMovidas': 42, 'tarjetasEliminadas': 19}","{'tarjetasCreadas': 131, 'tarjetasMovidas': 21, 'tarjetasEliminadas': 8}","{'tarjetasCreadas': 195, 'tarjetasMovidas': 16, 'tarjetasEliminadas': 20}","{'tarjetasCreadas': 136, 'tarjetasMovidas': 51, 'tarjetasEliminadas': 23}","{'tarjetasCreadas': 195, 'tarjetasMovidas': 54, 'tarjetasEliminadas': 35}","{'tarjetasCreadas': 113, 'tarjetasMovidas': 10, 'tarjetasEliminadas': 4}","{'tarjetasCreadas': 211, 'tarjetasMovidas': 63, 'tarjetasEliminadas': 28}","{'tarjetasCreadas': 209, 'tarjetasMovidas': 50, 'tarjetasEliminadas': 19}","{'tarjetasCreadas': 141, 'tarjetasMovidas': 14, 'tarjetasEliminadas': 6}","{'tarjetasCreadas': 118, 'tarjetasMovidas': 27, 'tarjetasEliminadas': 13}","{'tarjetasCreadas': 176, 'tarjetasMovidas'